# Entscheidungsbäume - Iris Blumen erkennen 🌸

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_16/decision_trees_iris.ipynb)

## Willkommen zu unserem Blumen-Experten!

Heute bauen wir einen **Entscheidungsbaum**, der automatisch verschiedene Iris-Blumen erkennen kann. Am Ende können wir ihm die Maße einer unbekannten Blume geben und er sagt uns genau, welche Art es ist - **und warum**!

### Was wir heute lernen:
1. 🌺 **Daten verstehen**: Welche Blumen haben wir?
2. 🌳 **Baum trainieren**: Wie lernt der Computer Entscheidungen?
3. 👀 **Visualisieren**: Den Entscheidungsbaum sehen und verstehen
4. 🛡️ **Overfitting vermeiden**: Nicht zu kompliziert werden
5. 🎯 **Vorhersagen erklären**: Warum diese Entscheidung?

**Lasst uns anfangen!** 🚀

In [ ]:
# Schritt 1: Alle benötigten Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Für schönere Plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("🎉 Alle Bibliotheken erfolgreich geladen!")
print("Wir sind bereit, unseren Blumen-Experten zu bauen!")

## 🌺 Schritt 1: Die Iris-Blumen kennenlernen

Der **Iris-Datensatz** ist ein Klassiker in der Datenwissenschaft. Er enthält Messungen von 150 Iris-Blumen aus drei verschiedenen Arten.

Petal und Sepal sind botanische Begriffe für Teile einer Blüte:

🌸 Sepal (Kelchblatt):

Deutsche Bezeichnung: Kelchblatt

Position: Äußere Schicht der Blüte

Funktion: Schützt die Knospe vor dem Aufblühen

Aussehen: Meist grün und derber/fester

Struktur: Umhüllt die inneren Blütenteile

🌺 Petal (Kronblatt):

Deutsche Bezeichnung: Kronblatt

Position: Innere Schicht der Blüte

Funktion: Lockt Bestäuber an (Bienen, Schmetterlinge)

Aussehen: Meist bunt/farbig und zarter

Struktur: Die "schönen" Blütenblätter, die wir sehen

In [ ]:
# Iris-Datensatz laden
iris = load_iris()
X = iris.data  # Die Messungen (Features)
y = iris.target  # Die Blumenarten (Labels)

# Als DataFrame für bessere Übersicht
df = pd.DataFrame(X, columns=iris.feature_names)
df['Blumenart'] = [iris.target_names[i] for i in y]

print("🌸 Iris-Datensatz erfolgreich geladen!")
print(f"📊 {len(df)} Blumen, {len(iris.feature_names)} Messungen pro Blume")
print(f"🏷️ Arten: {list(iris.target_names)}")
print(f"📏 Messungen: {list(iris.feature_names)}")

# Erste 5 Blumen anschauen
print("\n👀 Die ersten 5 Blumen in unserem Datensatz:")
display(df.head())

In [ ]:
# Verteilung der Blumenarten anschauen
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
art_counts = df['Blumenart'].value_counts()
plt.pie(art_counts.values, labels=art_counts.index, autopct='%1.0f%%', startangle=90)
plt.title('Verteilung der Blumenarten')

plt.subplot(1, 3, 2)
sns.boxplot(data=df, x='Blumenart', y='petal length (cm)')
plt.title('Blütenblattlänge pro Art')
plt.xticks(rotation=45)

plt.subplot(1, 3, 3)
sns.scatterplot(data=df, x='petal length (cm)', y='petal width (cm)', hue='Blumenart')
plt.title('Blütenblatt: Länge vs. Breite')

plt.tight_layout()
plt.show()

print("\n💡 Was sehen wir?")
print("   🔍 Alle drei Arten sind gleich häufig (je 50 Stück)")
print("   📏 Setosa hat die kürzesten Blütenblätter")
print("   🎯 Die Arten scheinen gut unterscheidbar zu sein!")

## 🌳 Schritt 2: Unseren ersten Entscheidungsbaum trainieren

Jetzt bauen wir unseren ersten "Blumen-Experten"! Wir teilen die Daten in **Training** (zum Lernen) und **Test** (zum ehrlichen Bewerten) auf.

In [ ]:
# Daten in Training und Test aufteilen
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,      # 30% für Test
    random_state=42,    # Für reproduzierbare Ergebnisse
    stratify=y          # Gleiche Verteilung in Train und Test
)

print(f"🚂 Training: {len(X_train)} Blumen")
print(f"🧪 Test: {len(X_test)} Blumen")
print(f"⚖️ Verhältnis: {len(X_train)/len(X_test):.1f}:1 (Train:Test)")

# Unseren ersten Baum erstellen (erstmal ohne Einschränkungen)
tree_unlimited = DecisionTreeClassifier(
    random_state=42,
    criterion='gini'  # Wie misst er "Unreinheit"?
)

print("\n🌳 Trainiere den unbegrenzten Baum...")
tree_unlimited.fit(X_train, y_train)

# Vorhersagen machen
y_pred = tree_unlimited.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✨ Training abgeschlossen!")
print(f"🎯 Genauigkeit auf Testdaten: {accuracy:.1%}")
print(f"🌿 Baumtiefe: {tree_unlimited.get_depth()} Ebenen")
print(f"🍃 Anzahl Blätter: {tree_unlimited.get_n_leaves()}")

## 👀 Schritt 3: Den Entscheidungsbaum visualisieren

Jetzt kommt das Spannende: Wir schauen uns an, **wie** unser Baum Entscheidungen trifft!

In [ ]:
# Den kompletten Baum visualisieren
plt.figure(figsize=(20, 12))
plot_tree(
    tree_unlimited,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    filled=True,        # Farbige Knoten
    rounded=True,       # Runde Ecken
    fontsize=10
)
plt.title('Unser erster Entscheidungsbaum (unbegrenzt)', fontsize=16, fontweight='bold')
plt.show()

print("\n🔍 So liest man den Baum:")
print("   📊 Jeder Knoten zeigt: Frage, Gini-Wert, Anzahl Proben, Verteilung")
print("   🎨 Farbe zeigt die Mehrheitsklasse in diesem Knoten")
print("   ⬅️➡️ Links = 'Ja' zur Frage, Rechts = 'Nein' zur Frage")
print("   🍃 Blätter (unten) = finale Entscheidungen")

In [ ]:
# Eine vereinfachte Version für bessere Lesbarkeit
tree_simple = DecisionTreeClassifier(
    max_depth=3,        # Maximal 3 Fragen hintereinander
    random_state=42
)

tree_simple.fit(X_train, y_train)
y_pred_simple = tree_simple.predict(X_test)
accuracy_simple = accuracy_score(y_test, y_pred_simple)

plt.figure(figsize=(15, 10))
plot_tree(
    tree_simple,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    filled=True,
    rounded=True,
    fontsize=12
)
plt.title(f'Vereinfachter Baum (max_depth=3) - Genauigkeit: {accuracy_simple:.1%}', 
          fontsize=16, fontweight='bold')
plt.show()

print(f"\n📈 Vergleich:")
print(f"   🌳 Unbegrenzter Baum: {accuracy:.1%} Genauigkeit, {tree_unlimited.get_depth()} Tiefe")
print(f"   ✂️ Vereinfachter Baum: {accuracy_simple:.1%} Genauigkeit, {tree_simple.get_depth()} Tiefe")
print(f"\n💡 Der einfachere Baum ist besser und viel verständlicher!")

## 🎯 Schritt 4: Einzelne Vorhersagen verfolgen

Lassen wir den Baum eine konkrete Blume klassifizieren und verfolgen seinen "Denkprozess"!

In [ ]:
# Eine Beispielblume aus dem Testset nehmen
beispiel_idx = 5
beispiel_blume = X_test[beispiel_idx]
wahre_art = iris.target_names[y_test[beispiel_idx]]
vorhergesagte_art = iris.target_names[tree_simple.predict([beispiel_blume])[0]]

print("🌸 Beispielblume:")
print(f"   📏 Kelchblattlänge: {beispiel_blume[0]:.1f} cm")
print(f"   📏 Kelchblattbreite: {beispiel_blume[1]:.1f} cm")
print(f"   📏 Blütenblattlänge: {beispiel_blume[2]:.1f} cm")
print(f"   📏 Blütenblattbreite: {beispiel_blume[3]:.1f} cm")
print(f"\n🏷️ Tatsächliche Art: {wahre_art}")
print(f"🤖 Vorhersage des Baums: {vorhergesagte_art}")
print(f"✅ Korrekt: {'Ja! 🎉' if wahre_art == vorhergesagte_art else 'Nein 😞'}")

# Wahrscheinlichkeiten für alle Klassen
probabilities = tree_simple.predict_proba([beispiel_blume])[0]
print(f"\n🎲 Sicherheit der Vorhersage:")
for i, prob in enumerate(probabilities):
    print(f"   {iris.target_names[i]}: {prob:.1%}")

In [ ]:
# Verwirrungsmatrix - wo macht der Baum Fehler?
from sklearn.metrics import ConfusionMatrixDisplay

plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_simple)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp.plot(cmap='Blues', values_format='d')
plt.title('Wo macht unser Baum Fehler?', fontsize=14, fontweight='bold')
plt.show()

print("\n📊 Interpretation der Matrix:")
print("   ✅ Diagonale = richtige Vorhersagen")
print("   ❌ Andere Felder = Verwechslungen")
print("   💡 Je dunkler das Blau, desto mehr Fälle")

# Detaillierter Bericht
print("\n📋 Detaillierter Klassifikationsbericht:")
print(classification_report(y_test, y_pred_simple, target_names=iris.target_names))

## 🛡️ Schritt 5: Overfitting vermeiden - Parameter experimentieren

Jetzt testen wir verschiedene Einstellungen und schauen, welche am besten funktionieren!

In [ ]:
# Verschiedene max_depth Werte testen
depths = range(1, 11)
train_accuracies = []
test_accuracies = []

print("🧪 Teste verschiedene Baumtiefen...")

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
    print(f"   Tiefe {depth}: Training {train_acc:.1%}, Test {test_acc:.1%}")

# Ergebnisse visualisieren
plt.figure(figsize=(10, 6))
plt.plot(depths, train_accuracies, 'o-', label='Training-Genauigkeit', color='green')
plt.plot(depths, test_accuracies, 's-', label='Test-Genauigkeit', color='red')
plt.xlabel('Maximale Baumtiefe')
plt.ylabel('Genauigkeit')
plt.title('Training vs. Test Genauigkeit bei verschiedenen Baumtiefen')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

best_depth = depths[np.argmax(test_accuracies)]
best_test_acc = max(test_accuracies)
print(f"\n🏆 Beste Tiefe: {best_depth} (Test-Genauigkeit: {best_test_acc:.1%})")

In [ ]:
# Verschiedene min_samples_leaf Werte testen
min_samples_values = [1, 2, 5, 10, 15, 20]
test_accs_samples = []

print("🧪 Teste verschiedene minimale Blattgrößen...")

for min_samples in min_samples_values:
    tree = DecisionTreeClassifier(
        max_depth=best_depth, 
        min_samples_leaf=min_samples, 
        random_state=42
    )
    tree.fit(X_train, y_train)
    test_acc = tree.score(X_test, y_test)
    test_accs_samples.append(test_acc)
    print(f"   Min. {min_samples} Proben/Blatt: {test_acc:.1%}")

plt.figure(figsize=(10, 6))
plt.plot(min_samples_values, test_accs_samples, 'o-', color='purple', linewidth=2, markersize=8)
plt.xlabel('Minimale Anzahl Proben pro Blatt')
plt.ylabel('Test-Genauigkeit')
plt.title('Einfluss der minimalen Blattgröße auf die Performance')
plt.grid(True, alpha=0.3)
plt.show()

best_min_samples = min_samples_values[np.argmax(test_accs_samples)]
print(f"\n🏆 Beste min_samples_leaf: {best_min_samples}")

## 🎨 Schritt 6: Entscheidungsgrenzen visualisieren

Schauen wir uns an, wie der Baum den "Blumenraum" in rechteckige Regionen aufteilt!

In [ ]:
# Optimal tuned tree erstellen
optimal_tree = DecisionTreeClassifier(
    max_depth=best_depth,
    min_samples_leaf=best_min_samples,
    random_state=42
)
optimal_tree.fit(X_train, y_train)

# Für 2D-Plot nehmen wir nur zwei Features
X_2d = X[:, [2, 3]]  # Blütenblattlänge und -breite
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(
    X_2d, y, test_size=0.3, random_state=42, stratify=y
)

tree_2d = DecisionTreeClassifier(
    max_depth=best_depth,
    min_samples_leaf=best_min_samples,
    random_state=42
)
tree_2d.fit(X_train_2d, y_train_2d)

# Entscheidungsgrenzen plotten
def plot_decision_regions(X, y, classifier, title):
    h = 0.01  # Schrittweite im Gitter
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Gleiche Farben für Entscheidungsgrenzen und Scatter-Punkte verwenden
    colors = ['red', 'blue', 'green']
    from matplotlib.colors import ListedColormap
    cmap = ListedColormap(colors)
    plt.contourf(xx, yy, Z, alpha=0.4, cmap=cmap)
    
    for i, color in enumerate(colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, 
                   label=iris.target_names[i], alpha=0.8, s=50)
    
    plt.xlabel('Blütenblattlänge (cm)')
    plt.ylabel('Blütenblattbreite (cm)')
    plt.title(title)
    plt.legend()

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plot_decision_regions(X_2d, y, tree_2d, 
                     f'Entscheidungsbaum (Tiefe {best_depth})')

# Zum Vergleich: Sehr einfacher Baum
simple_tree = DecisionTreeClassifier(max_depth=2, random_state=42)
simple_tree.fit(X_train_2d, y_train_2d)

plt.subplot(1, 2, 2)
plot_decision_regions(X_2d, y, simple_tree, 'Einfacher Baum (Tiefe 2)')

plt.tight_layout()
plt.show()

print("\n🎨 Was sehen wir?")
print("   📐 Entscheidungsbäume teilen immer in Rechtecke auf")
print("   🎯 Jede Farbe = eine Region für eine Blumenart")
print("   🔍 Komplexere Bäume → mehr kleine Rechtecke")
print("   ⚖️ Einfachere Bäume → weniger, größere Rechtecke")

## 🏆 Schritt 7: Feature-Wichtigkeit - Was ist am wichtigsten?

Lassen wir uns vom Baum sagen, welche Messungen am wichtigsten für die Klassifikation sind!

In [ ]:
# Feature-Wichtigkeit aus unserem optimalen Baum
feature_importance = optimal_tree.feature_importances_
feature_names = iris.feature_names

# Sortiert nach Wichtigkeit
indices = np.argsort(feature_importance)[::-1]

print("🏆 Ranking der Feature-Wichtigkeit:")
for i, idx in enumerate(indices):
    print(f"   {i+1}. {feature_names[idx]}: {feature_importance[idx]:.3f}")

# Visualisierung
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance[indices], 
        color=['gold', 'silver', '#CD7F32', 'lightgray'])
plt.xlabel('Features')
plt.ylabel('Wichtigkeit')
plt.title('Feature-Wichtigkeit im Entscheidungsbaum')
plt.xticks(range(len(feature_importance)), 
           [feature_names[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()

print(f"\n💡 Interpretation:")
most_important = feature_names[indices[0]]
print(f"   🥇 '{most_important}' ist am wichtigsten für die Klassifikation")
print(f"   📊 Die Werte summieren sich zu: {sum(feature_importance):.3f}")
print(f"   🎯 Höhere Werte = wichtiger für Entscheidungen")

## 🔬 Schritt 8: Experimentierbereich - Jetzt seid ihr dran!

Hier könnt ihr selbst experimentieren und verschiedene Einstellungen ausprobieren!

In [ ]:
# Experimentier-Zelle: Probiert verschiedene Parameter aus!

print("🧪 Experimentierbereich - Ändert die Parameter und schaut, was passiert!")
print("\n🎛️ Parameter zum Experimentieren:")
print("   - max_depth: 1, 2, 3, 5, 10, None (unbegrenzt)")
print("   - min_samples_leaf: 1, 5, 10, 20")
print("   - criterion: 'gini', 'entropy'")
print("   - min_samples_split: 2, 5, 10")

# HIER EXPERIMENTIEREN!
experiment_tree = DecisionTreeClassifier(
    max_depth=3,              # 🔧 Ändert diese Werte!
    min_samples_leaf=5,       # 🔧 Ändert diese Werte!
    criterion='gini',         # 🔧 Probiert 'entropy'!
    min_samples_split=10,     # 🔧 Neue Parameter hinzufügen!
    random_state=42
)

experiment_tree.fit(X_train, y_train)
experiment_pred = experiment_tree.predict(X_test)
experiment_acc = accuracy_score(y_test, experiment_pred)

print(f"\n📊 Euer Experiment:")
print(f"   🎯 Test-Genauigkeit: {experiment_acc:.1%}")
print(f"   🌿 Baumtiefe: {experiment_tree.get_depth()}")
print(f"   🍃 Anzahl Blätter: {experiment_tree.get_n_leaves()}")

# Schnelle Visualisierung
if experiment_tree.get_depth() <= 4:  # Nur kleine Bäume visualisieren
    plt.figure(figsize=(15, 8))
    plot_tree(experiment_tree, feature_names=iris.feature_names, 
              class_names=iris.target_names, filled=True, rounded=True, fontsize=10)
    plt.title(f'Euer Experiment-Baum (Genauigkeit: {experiment_acc:.1%})', 
              fontsize=14, fontweight='bold')
    plt.show()
else:
    print("   🌳 Baum zu groß für Visualisierung (Tiefe > 4)")

## 🎓 Zusammenfassung - Was haben wir gelernt?

### ✅ Das haben wir geschafft:

1. **🌺 Daten verstanden**: 150 Iris-Blumen, 3 Arten, 4 Messungen
2. **🌳 Entscheidungsbaum trainiert**: Vom einfachen bis zum optimierten Modell
3. **👀 Visualisiert**: Gesehen, wie der Baum "denkt"
4. **🛡️ Overfitting vermieden**: Richtige Parameter gefunden
5. **🎯 Vorhersagen erklärt**: Verstanden, warum bestimmte Entscheidungen getroffen werden
6. **🏆 Feature-Wichtigkeit**: Gelernt, welche Messungen am wichtigsten sind

### 🔑 Wichtigste Erkenntnisse:

**Vorteile von Entscheidungsbäumen:**
- ✅ **Erklärbar**: Jede Entscheidung ist nachvollziehbar
- ✅ **Keine Datenvorbearbeitung**: Rohe Daten funktionieren
- ✅ **Verschiedene Datentypen**: Zahlen und Kategorien zusammen
- ✅ **Nichtlineare Muster**: Kann komplexe Beziehungen finden

**Herausforderungen:**
- ⚠️ **Overfitting**: Ohne Kontrolle werden sie zu komplex
- ⚠️ **Instabilität**: Kleine Datenänderungen → großer Unterschied
- ⚠️ **Rechteckige Grenzen**: Nur achsenparallele Teilungen

### 🚀 Nächster Schritt könnte sein:

**Random Forest**: Viele Bäume = stabilere Vorhersagen

### 🎯 Praktische Tipps:

- **Startet einfach**: Erst kleine `max_depth`, dann erweitern
- **Visualisiert immer**: Bäume sind zum Anschauen da, zumindest wenn sie nicht zu groß sind!
- **Cross-Validation oder Train-Val-Test-Split**: Für zuverlässige Parameterauswahl
- **Feature-Wichtigkeit**: Hilft beim Daten verstehen

**🎉 Herzlichen Glückwunsch! Ihr habt erfolgreich euren ersten intelligenten Blumen-Experten gebaut!**